In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


#selenium


# In[4]:


pip install selenium


# In[ ]:


#libraries


# In[5]:


import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
from lxml import etree


# In[47]:


import pandas as pd
from datetime import datetime


# In[48]:


URL_P1 = 'https://www.youtube.com/@UnfoldDataScience/videos'


# In[13]:


Driver_Path = r"C:\Users\omert\OneDrive\Desktop\chromedriver-win64\chromedriver.exe"


# In[14]:


#HELPER FUNCTIONS
import time
from selenium.webdriver.common.keys import Keys

def scroll_down(scrolls, halfpage=False, sleep=0):
    """
    Scroll down a webpage using the Selenium WebDriver.

    Parameters:
    - scrolls: Number of times to scroll down.
    - halfpage: If True, scroll down by half a page; otherwise, scroll to the end of the page.
    - sleep: Optional sleep time (in seconds) between scrolls.

    Note: You should have the 'driver' variable set to your Selenium WebDriver instance before using this function.
    """
    while scrolls > 0:
        # Check if scrolling by half a page or to the end of the page
        if halfpage:
            driver.find_element('tag name', 'body').send_keys(Keys.PAGE_DOWN)
        else:
            driver.find_element('tag name', 'body').send_keys(Keys.END)

        scrolls -= 1

        # Sleep for the specified duration (if provided)
        if sleep > 0:
            time.sleep(sleep)

        # Print the number of scrolls left for tracking
        print("Scrolls left: ", scrolls)


# In[ ]:


def GetVidData(URL):
    driver.get(URL)
    time.sleep(1)
    #press the mute button
    scroll_down(1, sleep=3, halfpage=True)

    name = driver.find_elements(By.XPATH, '//*[@id="title"]/h1/yt-formatted-string')
    views = driver.find_elements(By.XPATH, '//*[@id="info"]/span[1]')
    likes = driver.find_elements(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button/div[2]/span')
    date = driver.find_elements(By.XPATH, '//*[@id="info"]/span[3]')
    comments = driver.find_elements(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[1]')

    return name, views, likes, date, comments


# In[22]:


# Create a WebDriver service
service = Service(Driver_Path)

# Initialize a Chrome WebDriver instance
driver = webdriver.Chrome(service=service)

# URL of the webpage you want to scrape
URL_P1 = 'https://example.com'

# Open the webpage in the browser
driver.get(URL_P1)

# Sleep for 10 seconds to allow the page to load
time.sleep(10)

# Scroll down the page 15 times with a 3-second delay between scrolls
scroll_down(15, sleep=3, halfpage=False)

# Initialize lists to store video data
video_likes = []
video_views = []
video_dates = []
video_names = []
video_comments = []

# Find all video links on the page using XPath
video_link = driver.find_elements(By.XPATH, '//*[@id="video-title-link"]')

# Extract the href attribute from video links and store them in a list
video_links = [link.get_attribute('href') for link in video_link]

# Loop through the first 10 video links
for link in video_links[:10]:
    # Call a function to extract video data from each link
    name, views, likes, date, comments = GetVidData(link)

    # Append extracted data to respective lists
    video_names.append(name[0].text)
    video_views.append(views[0].text)
    video_likes.append(likes[0].text)
    video_dates.append(date[0].text)
    video_comments.append(comments[0].text)

# Close the WebDriver
driver.close()

# Create a DataFrame with the extracted video data
data = {
    'Video Name': video_names,
    'No.Of Views': video_views,
    'Date Published': video_dates,
    'No.Of Likes': video_likes,
    'No.Of Comments': video_comments
}

# Convert the DataFrame to an Excel file and save it as 'V1.xlsx' without an index
data = pd.DataFrame(data).to_excel('V1.xlsx', index=False)


# In[26]:


df = pd.read_excel('V1.xlsx')
df


# In[30]:


from datetime import datetime
from dateutil.relativedelta import relativedelta

def GetDate(date):
    # Split the input string into components: amount, type_, and _
    date = date.strip().split(' ')
    amount, type_, _ = date

    # Get the current date
    today = datetime.today()

    # Check the type of time unit (days, weeks, months, or years)
    if type_ == 'days' or type_ == 'day':
        # Subtract the specified number of days from the current date
        date = today - relativedelta(days=int(amount))
    elif type_ == 'weeks' or type_ == 'week':
        # Subtract the specified number of weeks from the current date
        date = today - relativedelta(weeks=int(amount))
    elif type_ == 'months' or type_ == 'month':
        # Subtract the specified number of months from the current date
        date = today - relativedelta(months=int(amount))
    elif type_ == 'years' or type_ == 'year':
        # Subtract the specified number of years from the current date
        date = today - relativedelta(years=int(amount))

    # Return the resulting date as a date object
    return date.date()


# In[31]:


df['Date Published'] = df['Date Published'].apply(GetDate)


# In[32]:


df['Date Published']


# In[44]:


df['No.Of Views']
def convert_views(views):
    if views <= 6.8:
        return int(views * 1000)
    else:
        return int(views)

# Apply the conversion function to the 'No.Of Views' column
df['No.Of Views (Numeric)'] = df['No.Of Views'].apply(convert_views)


# In[45]:


df['No.Of Views (Numeric)']


# In[51]:


def avgViews30days(df):
    df['Date Published'] = pd.to_datetime(df['Date Published'])
    currDate = datetime.now()
    #cur-dat pub
    df['Days Ago'] = (currDate - df['Date Published']).dt.days
    #after getting data in form of days
    #select the rows within 30
    withInMonth = df[df['Days Ago'] <= 30]
    #only summing their respective views
    viewsWithInMonth = withInMonth['No.Of Views (Numeric)'].sum()
    #count totaltal
    vidCount = withInMonth.shape[0]
    #avg
    avgViews = viewsWithInMonth / vidCount

    return avgViews

#funct call
average_views = avgViews30days(df)

#ans
print('Average Views Count per Video in the Last 30 Days:', average_views)


# In[53]:


#Parameters     df    likes  comments
def findCorr(dataframe, col1, col2):
  #FINFING CORR
    correlation = dataframe[col1].corr(dataframe[col2])
    return correlation


#calling funct
correlation = findCorr(df, 'No.Of Likes', 'No.Of Comments')

print("Correlation between Likes and Comments: ",correlation)


# In[54]:


# Convert the 'Date Published' column to datetime objects
df['Date Published'] = pd.to_datetime(df['Date Published'])

# Extract the day of the week and create a new column
df['Day of Week'] = df['Date Published'].dt.day_name()

# Find the most common day of the week
most_common_day = df['Day of Week'].mode()[0]

print(f'Most Common Day for Video Uploads: {most_common_day}')


# In[55]:


#parameter    df        col
def weekDay(dataframe, col):
    #date col->date time
    dataframe[col] = pd.to_datetime(dataframe[col])
    # from date column fetch weeek
    dataframe['dayOfWeek'] = dataframe[col].dt.day_name()
    #common day
    #the most common day will be sorted obv in top order
    #fetch that
    commonWeekDat = dataframe['dayOfWeek'].mode()[0]

    return commonWeekDat


# calling funct
commonWeekDat = weekDay(df, 'Date Published')

print('Most Common Day for Video Uploads:',commonWeekDat)


# In[61]:


#finding outliers with the same methodology
#as used above
#parameters       df   views
def killOutliers(df, colName):
    # FIND Q1 AND thirdQuartile
    firstQuartile = df[colName].quantile(0.25)
    thirdQuartile = df[colName].quantile(0.75)
    #IQR
    InterQuartileRange = thirdQuartile - firstQuartile
    #LB AND UP
    LB = firstQuartile - 1.5 * InterQuartileRange
    UP = thirdQuartile + 1.5 * InterQuartileRange
    #REST WILL BE OUTLIERS
    outliers = df[(df[colName] < LB) | (df[colName] > UP)]

    return outliers


# FUNCTION
outliers = killOutliers(df, 'No.Of Views')

# Print the outliers
print('Outliers in Views Count:')
print(outliers)

